In [ ]:
# initialization for my classroom
import os
from datetime import datetime as dt

def logfile(user=os.environ.get('JUPYTERHUB_USER') or 'jovyan'):
    prefix='/srv'
    if os.path.isdir(prefix) and os.access(prefix, os.W_OK):
        prefix+=('/'+user)
        if not os.path.isdir(prefix):
            os.makedirs(prefix)
    else:
        prefix='.'
    return prefix+'/'+dt.now().strftime('%Y%m%d')+'.log'

path=logfile()
#%logstop
%logstart -otq $path append

# [python - cannot override sys.excepthook - Stack Overflow](https://stackoverflow.com/questions/1261668/cannot-override-sys-excepthook/28758396)
# https://github.com/ipython/ipython/blob/e6432249582e05f438303ce73d082a0351bb383e/IPython/core/interactiveshell.py#L1952

import sys
import traceback
import IPython

try:
    _showtraceback
except NameError:
    _showtraceback=IPython.core.interactiveshell.InteractiveShell.showtraceback

try:
    _showsyntaxerror
except NameError:
    _showsyntaxerror=IPython.core.interactiveshell.InteractiveShell.showsyntaxerror

import logging
logging.basicConfig(filename=path.replace('.log','-exc.log'), format='%(asctime)s %(message)s', level=logging.ERROR, force=True)

import sys
import traceback
import IPython

def showtraceback(self, *args, **kwargs):
    etype, value, tb = self._get_exc_info(kwargs.get('exc_tuple'))
    stb = self.InteractiveTB.structured_traceback(
        etype, value, tb, tb_offset=kwargs.get('tb_offset'))
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showtraceback(self, *args, **kwargs)

def showsyntaxerror(self, *args, **kwargs):
    etype, value, last_traceback = self._get_exc_info()
    elist = traceback.extract_tb(last_traceback) if kwargs.get('running_compiled_code') else []
    stb = self.SyntaxTB.structured_traceback(etype, value, elist)
    logging.error(os.environ.get('JUPYTERHUB_USER') or 'jovyan')
    logging.error(self.InteractiveTB.stb2text(stb))
    _showsyntaxerror(self, *args, **kwargs)

IPython.core.interactiveshell.InteractiveShell.showtraceback = showtraceback
IPython.core.interactiveshell.InteractiveShell.showsyntaxerror = showsyntaxerror

# 課題-3) Plimpton 322 に記載された数表を補完する

In [ ]:
import math
import itertools

In [ ]:
def generate_p(i_max=2, j_max=2, k_max=2):
    l = []
    for i in range(i_max):
        for j in range(j_max):
            for k in range(k_max):
                l.append(2**i * 3**j * 5**k)
    return sorted(l)

generate_p()

In [ ]:
def plimpton322(i_max=2, j_max=2, k_max=2):
    pq = []
    for p, q in itertools.product(generate_p(i_max,j_max,k_max), repeat=2):
        if p > q and math.gcd(p,q) ==1:
            if p/q >= 9/5 and p/q <= 12/5:
                pq.append((p,q))

    return sorted(pq, reverse=True, key=lambda x: x[0]/x[1])

In [ ]:
plimpton322()

In [ ]:
plimpton322(4,3,2)

In [ ]:
import numpy as np
import pandas as pd

l = []
for p, q in plimpton322(4,3,2):
    a = 2 * p * q
    b = p**2 - q**2
    d = p**2 + q**2
    l.append(((d/a)**2, b, d))
l

In [ ]:
df = pd.DataFrame(l, columns=['(d/a)**2', 'b', 'd'])
df

In [ ]:
import pandas as pd
import requests

res = requests.get("https://en.wikipedia.org/wiki/Plimpton_322").text

df2=pd.read_html(res)[1]
df2

## YBC 6967 に基づく解釈

$c$を定数とする二次方程式

$$
x - \frac{1}{x} = c
$$

について、
$v_{1} = \frac{c}{2}$、
$v_{3} = 1 + {v_{1}}^2 = 1 + (\frac{c}{2})^2$、
$v_{4} = \sqrt{v_{3}} = \sqrt{1 + (\frac{c}{2})^2}$ と置くと、次の連立方程式が成り立つ:

$$
  \begin{align}
    x &= v_{4} + v_{1} \\
    \frac{1}{x} &= v_{4} - v_{1}
  \end{align}
$$

補足) これまで出てきた関係式をすべて一覧表に追加 

|#1 |#2 |#3 |
|:-:|:-:|:-:|
|$$\left(\frac{d}{a}\right)^2$$|$$b$$      |$$d$$      |
|$$\frac{1}{4}\left(\frac{p}{q}+\frac{q}{p}\right)^2$$|$$p^2-q^2$$|$$p^2+q^2$$|
|$$1+{v_{1}}^2 = {v_{4}}^2$$|$$a\cdot v_{1}$$|$$a\cdot v_{4}$$|
|$$1+\left(\frac{c}{2}\right)^2$$|$$\frac{a}{2}\left(x-\frac{1}{x}\right)$$|$$\frac{a}{2}\left(x+\frac{1}{x}\right)$$|

補足2) Plimpton 322の左側の欠落を想像してみる

|.  |.  |.  |.  |
|:-:|:-:|:-:|:-:|
|$$\frac{d+b}{a}$$|$$\frac{d-b}{a}$$|$$2 \frac{b}{a}$$|$$a$$|
|$$\frac{p}{q}$$|$$\frac{q}{p}$$|$$\frac{p}{q}-\frac{q}{p}$$|$$2qp$$|
|$$v_{4}+v_{1}$$|$$v_{4}-v_{1}$$|$$2\cdot v_{1}$$|$$a$$|
|$$x$$|$$\frac{1}{x}$$|$$c=x-\frac{1}{x}$$|$$a$$|

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
l = []
for p, q in plimpton322(4,3,2):
    a = 2 * p * q
    b = p**2 - q**2
    d = p**2 + q**2
    l.append((Fraction(b+d,a), Fraction(d-b,a), Fraction(2*b,a), a, b, d))
l

In [ ]:
df = pd.DataFrame(l, columns=['x', '1/x', 'c', 'a', 'b', 'd'])
df

In [ ]:
import numpy as np
import pandas as pd
from fractions import Fraction
l = []
for p, q in plimpton322(7,6,4):
    a = 2 * p * q
    b = p**2 - q**2
    d = p**2 + q**2
    l.append((Fraction(b+d,a), Fraction(d-b,a), Fraction(2*b,a), a, b, d))
l;

In [ ]:
df = pd.DataFrame(l, columns=['x', '1/x', 'c', 'a', 'b', 'd'])
df

---
以下、採点用のセルにつき編集できない:

In [ ]:
from nose.tools import assert_equal, assert_true